Loading data
------------

In [ ]:
display(dbutils.fs.ls("/datasets/group12/"))

  

[TABLE]

  

Load parquet file

In [ ]:
val df = spark.read.parquet("dbfs:/datasets/group12/analysis/*.parquet")

display(df)


  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

Load csv file

-&gt; %scala //if want to load csv

val file*location = "/datasets/group12/20*12*04*10*47*08.csv" val
file\_type = "csv"

// CSV options val infer*schema = "true" val first*row*is*header =
"true" val delimiter = ","

// The applied options are for CSV files. For other file types, these
will be ignored. val df = spark.read.format(file*type)
.option("inferSchema", infer*schema) .option("header",
first*row*is*header) .option("sep", delimiter) .load(file*location)

display(df)

Number of data

In [ ]:
df.count()

  

>     res19: Long = 60544

  

Missing Features in data due to multiple web resource

In [ ]:
import org.apache.spark.sql.functions._

for (c <- df.columns) {
  println(c + ": " + df.filter(col(c).isNull).count())
}

  

>     iso_code: 316
>     continent: 633
>     location: 0
>     date: 0
>     total_cases: 8739
>     new_cases: 123
>     new_cases_smoothed: 1079
>     total_deaths: 16597
>     new_deaths: 123
>     new_deaths_smoothed: 1079
>     total_cases_per_million: 9040
>     new_cases_per_million: 439
>     new_cases_smoothed_per_million: 1390
>     total_deaths_per_million: 16885
>     new_deaths_per_million: 439
>     new_deaths_smoothed_per_million: 1390
>     reproduction_rate: 20494
>     icu_patients: 55821
>     icu_patients_per_million: 55821
>     hosp_patients: 54834
>     hosp_patients_per_million: 54834
>     weekly_icu_admissions: 60083
>     weekly_icu_admissions_per_million: 60083
>     weekly_hosp_admissions: 59796
>     weekly_hosp_admissions_per_million: 59796
>     total_tests: 36667
>     new_tests: 36819
>     total_tests_per_thousand: 36667
>     new_tests_per_thousand: 36819
>     new_tests_smoothed: 34084
>     new_tests_smoothed_per_thousand: 34084
>     tests_per_case: 35333
>     positive_rate: 35924
>     tests_units: 60544
>     stringency_index: 7787
>     population: 316
>     population_density: 1903
>     median_age: 3160
>     aged_65_older: 4115
>     aged_70_older: 3476
>     gdp_per_capita: 3167
>     extreme_poverty: 21265
>     cardiovasc_death_rate: 2598
>     diabetes_prevalence: 1903
>     female_smokers: 15569
>     male_smokers: 16201
>     handwashing_facilities: 30455
>     hospital_beds_per_thousand: 7662
>     life_expectancy: 632
>     human_development_index: 2852
>     import org.apache.spark.sql.functions._

  

Preprocessing
-------------

### 0. filter out data of HongKong and unknown location

Here shows HK does not have meaningful value and there is one unknown
international location in data.

In [ ]:
display(df.filter($"location"==="Hong Kong" || $"iso_code".isNull)) //HK data iteself is not complete for all dates, and all available data is null! HAVE TO FILTER IT OUT COMPLETELY

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

190 valid countries data to continue

In [ ]:
val df_filteredLocation = df.filter($"iso_code"=!="HKG").filter($"iso_code".isNotNull)
display(df_filteredLocation.select($"location").distinct()) // 190 valid countries 

  

[TABLE]

Truncated to 30 rows

  

Fill missing continent value for World aggregate data NOTE: it will be
filled as "World"

In [ ]:
display(df_filteredLocation.where($"continent".isNull))

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
val df_fillContinentNull = df_filteredLocation.na.fill("World",Array("continent"))
display(df_fillContinentNull)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
df_fillContinentNull.count()

  

>     res27: Long = 60158

In [ ]:
import org.apache.spark.sql.functions._

for (c <- df_fillContinentNull.columns) {
  println(c + ": " + df_fillContinentNull.filter(col(c).isNull).count())
}

  

>     iso_code: 0
>     continent: 0
>     location: 0
>     date: 0
>     total_cases: 8654
>     new_cases: 53
>     new_cases_smoothed: 1004
>     total_deaths: 16499
>     new_deaths: 53
>     new_deaths_smoothed: 1004
>     total_cases_per_million: 8654
>     new_cases_per_million: 53
>     new_cases_smoothed_per_million: 1004
>     total_deaths_per_million: 16499
>     new_deaths_per_million: 53
>     new_deaths_smoothed_per_million: 1004
>     reproduction_rate: 20108
>     icu_patients: 55435
>     icu_patients_per_million: 55435
>     hosp_patients: 54448
>     hosp_patients_per_million: 54448
>     weekly_icu_admissions: 59697
>     weekly_icu_admissions_per_million: 59697
>     weekly_hosp_admissions: 59410
>     weekly_hosp_admissions_per_million: 59410
>     total_tests: 36294
>     new_tests: 36433
>     total_tests_per_thousand: 36294
>     new_tests_per_thousand: 36433
>     new_tests_smoothed: 33753
>     new_tests_smoothed_per_thousand: 33753
>     tests_per_case: 34947
>     positive_rate: 35538
>     tests_units: 60158
>     stringency_index: 7471
>     population: 0
>     population_density: 1587
>     median_age: 2844
>     aged_65_older: 3799
>     aged_70_older: 3160
>     gdp_per_capita: 2851
>     extreme_poverty: 20879
>     cardiovasc_death_rate: 2212
>     diabetes_prevalence: 1587
>     female_smokers: 15183
>     male_smokers: 15815
>     handwashing_facilities: 30069
>     hospital_beds_per_thousand: 7276
>     life_expectancy: 316
>     human_development_index: 2536
>     import org.apache.spark.sql.functions._

  

### 1. filter dates only from 2020-01-23 (to ensure all countries having 316 days logging)

In [ ]:
display(df_fillContinentNull.select($"date",$"iso_code").groupBy($"iso_code").count())  // some country starts logging data earlier

  

[TABLE]

Truncated to 30 rows

In [ ]:
val df_filtered_date = df_fillContinentNull.filter($"date">"2020-01-22")

  

>     df_filtered_date: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [iso_code: string, continent: string ... 48 more fields]

In [ ]:
display(df_filtered_date.select($"date",$"iso_code").groupBy($"iso_code").count())  // all countries have 316 days logging

  

[TABLE]

Truncated to 30 rows

  

### 2. Fill missing value for total*cases, total*deaths, new*cases*smoothed, new*deaths*smoothed

In [ ]:
display(df_filtered_date.select($"date",$"iso_code", $"total_cases", $"total_deaths", $"new_cases", $"new_deaths", $"new_cases_smoothed", $"new_deaths_smoothed").filter($"new_cases_smoothed".isNull || $"new_deaths_smoothed".isNull))

  

[TABLE]

Truncated to 30 rows

  

All missing data of new*cases*smoothed and new*deaths*smoothed from
early, so just fill with 0

In [ ]:
display(df_filtered_date.select($"date",$"iso_code", $"total_cases", $"total_deaths", $"new_cases", $"new_deaths", $"new_cases_smoothed", $"new_deaths_smoothed")
        .filter($"new_cases_smoothed".isNull || $"new_deaths_smoothed".isNull).select($"date").distinct())

  

[TABLE]

In [ ]:
val df_fillNullForSmooth = df_filtered_date.na.fill(0,Array("new_cases_smoothed"))
                           .na.fill(0,Array("new_deaths_smoothed"))
display(df_fillNullForSmooth)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

Fill total*deaths and total*cases null value

Strictly, when new*cases is always 0, total*cases could be imputed as 0.
The same apply to total\_deaths

In [ ]:
val df_NULL_total_cases = df_fillNullForSmooth.select($"date",$"iso_code", $"total_cases", $"total_deaths", $"new_cases", $"new_deaths", $"new_cases_smoothed", $"new_deaths_smoothed")
                          .filter($"total_cases".isNull)


display(df_NULL_total_cases.filter($"new_cases"===0).groupBy("iso_code").count())

  

[TABLE]

Truncated to 30 rows

  

When total*case is Null, all previous new*cases is always 0.

In [ ]:
df_NULL_total_cases.filter($"total_cases".isNull).groupBy("iso_code").count().except(df_NULL_total_cases.filter($"new_cases"===0).groupBy("iso_code").count()).show() // When total_case is Null, all new_cases is always 0

  

>     +--------+-----+
>     |iso_code|count|
>     +--------+-----+
>     +--------+-----+

In [ ]:
val df_fillNullForTotalCases = df_fillNullForSmooth.na.fill(0, Array("total_cases"))
                               
display(df_fillNullForTotalCases)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
val df_NULL_total_death = df_fillNullForTotalCases.select($"date",$"iso_code", $"total_cases", $"total_deaths", $"new_cases", $"new_deaths", $"new_cases_smoothed", $"new_deaths_smoothed")
                          .filter($"total_deaths".isNull)


display(df_NULL_total_death.filter($"new_deaths"===0).groupBy("iso_code").count().sort())

  

[TABLE]

Truncated to 30 rows

  

If total*deaths is Null when all new*deaths is always 0, then we could
simply assign 0 for NULL, otherwise need to investigate more.

Three countries (ISL, PNG, SVK) have abnormal correction on new\_cases
data.

In [ ]:
val abnormal_countries = df_NULL_total_death.filter($"total_deaths".isNull).groupBy("iso_code").count().except(df_NULL_total_death.filter($"new_deaths"===0).groupBy("iso_code").count())
abnormal_countries.show()
df_NULL_total_death.filter($"new_deaths"===0).groupBy("iso_code").count().except(df_NULL_total_death.filter($"total_deaths".isNull).groupBy("iso_code").count()).show()


  

>     +--------+-----+
>     |iso_code|count|
>     +--------+-----+
>     |     PNG|  186|
>     |     SVK|   65|
>     |     ISL|   54|
>     +--------+-----+
>
>     +--------+-----+
>     |iso_code|count|
>     +--------+-----+
>     |     PNG|  185|
>     |     SVK|   64|
>     |     ISL|   52|
>     +--------+-----+
>
>     abnormal_countries: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [iso_code: string, count: bigint]

  

show abnormal death correction

In [ ]:
display(df_fillNullForSmooth.filter($"iso_code"==="ISL").sort("date").filter($"date">"2020-03-13" && $"date"<"2020-03-22")) // death data correction between 2020-03-14 and 2020-03-21, total_deaths -> all 0, new_deaths -> all 0, new_deaths_smoothed -> all 0

  

[TABLE]

Truncated to 12 cols

In [ ]:
display(df_fillNullForSmooth.filter($"iso_code"==="PNG").sort("date").filter($"date">"2020-07-19" && $"date"<"2020-07-24" )) // death data correction between 2020-07-20 and 2020-07-22, total_deaths -> all 0, new_deaths -> all 0, new_deaths_smoothed -> all 0

  

[TABLE]

Truncated to 12 cols

In [ ]:
display(df_fillNullForSmooth.filter($"iso_code"==="SVK").sort("date").filter($"date">"2020-03-16" && $"date"<"2020-03-23")) // death data correction between 2020-03-18 and 2020-03-22, total_deaths -> all 0, new_deaths -> all 0, new_deaths_smoothed -> all 0

  

[TABLE]

Truncated to 12 cols

  

Correct new\_deaths correction back to 0

In [ ]:
val df_fillNullForTotalDeathsSpecial = df_fillNullForTotalCases.withColumn("total_deaths_correct", 
                                        when(col("iso_code").equalTo("ISL")&&(col("date")>"2020-03-13" && col("date")<"2020-03-22"),0)
                                       .when(col("iso_code").equalTo("PNG")&&(col("date")>"2020-07-19" && col("date")<"2020-07-23"),0)
                                       .when(col("iso_code").equalTo("SVK")&&(col("date")>"2020-03-17" && col("date")<"2020-03-23"),0).otherwise(col("total_deaths")))
                            .withColumn("new_deaths_correct", 
                                        when(col("iso_code").equalTo("ISL")&&(col("date")>"2020-03-13" && col("date")<"2020-03-22"),0)
                                       .when(col("iso_code").equalTo("PNG")&&(col("date")>"2020-07-19" && col("date")<"2020-07-23"),0)
                                       .when(col("iso_code").equalTo("SVK")&&(col("date")>"2020-03-17" && col("date")<"2020-03-23"),0).otherwise(col("new_deaths")))
                            .withColumn("new_deaths_smoothed_correct", 
                                        when(col("iso_code").equalTo("ISL")&&(col("date")>"2020-03-13" && col("date")<"2020-03-22"),0)
                                       .when(col("iso_code").equalTo("PNG")&&(col("date")>"2020-07-19" && col("date")<"2020-07-23"),0)
                                       .when(col("iso_code").equalTo("SVK")&&(col("date")>"2020-03-17" && col("date")<"2020-03-23"),0).otherwise(col("new_deaths_smoothed")))



  

>     df_fillNullForTotalDeathsSpecial: org.apache.spark.sql.DataFrame = [iso_code: string, continent: string ... 51 more fields]

  

Expect to see an empty table, so correction is right

In [ ]:
val df_NULL_total_death_ = df_fillNullForTotalDeathsSpecial.select($"date",$"iso_code", $"total_cases", $"total_deaths_correct", $"new_cases", $"new_deaths_correct", $"new_cases_smoothed", $"new_deaths_smoothed_correct")
                          .filter($"total_deaths_correct".isNull)


df_NULL_total_death_.filter($"total_deaths_correct".isNull).groupBy("iso_code").count().except(df_NULL_total_death_.filter($"new_deaths_correct"===0).groupBy("iso_code").count()).show()

  

>     +--------+-----+
>     |iso_code|count|
>     +--------+-----+
>     +--------+-----+
>
>     df_NULL_total_death_: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: string, iso_code: string ... 6 more fields]

  

fill rest NULL value for total\_death.

In [ ]:
val df_fillNullForTotalDeaths = df_fillNullForTotalDeathsSpecial
                                  .drop("new_deaths", "total_deaths", "new_deaths_smoothed") // drop old column to rename
                                  .withColumnRenamed("new_deaths_correct","new_deaths")
                                  .withColumnRenamed("total_deaths_correct","total_deaths")
                                  .withColumnRenamed("new_deaths_smoothed_correct","new_deaths_smoothed")
                                  .na.fill(0, Array("total_deaths"))
                                  .select(df.columns.head, df.columns.tail: _*)
display(df_fillNullForTotalDeaths)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

### All first 10 column is clean now!

(All code above is for illustration, for processing just run cell below
)

In [ ]:

// filter unknow and HK data
val df_filteredLocation = df.filter($"iso_code"=!="HKG").filter($"iso_code".isNotNull)

// fill missing continent value for World data
val df_fillContinentNull = df_filteredLocation.na.fill("World",Array("continent")).cache
df_filteredLocation.unpersist()

// filter date before 2020-01-23
val df_filtered_date = df_fillContinentNull.filter($"date">"2020-01-22").cache
df_fillContinentNull.unpersist()

// fill missing for new_cases_smoothed and new_deaths_smoothed
val df_fillNullForSmooth = df_filtered_date.na.fill(0,Array("new_cases_smoothed"))
                           .na.fill(0,Array("new_deaths_smoothed"))
                           .cache
df_filtered_date.unpersist()

// fill missing for total_cases
val df_fillNullForTotalCases = df_fillNullForSmooth.na.fill(0, Array("total_cases")).cache
df_fillNullForSmooth.unpersist()

// correct total_deaths, new_deaths, new_deaths_smoothed
val df_fillNullForTotalDeathsSpecial = df_fillNullForTotalCases.withColumn("total_deaths_correct", 
                                        when(col("iso_code").equalTo("ISL")&&(col("date")>"2020-03-13" && col("date")<"2020-03-22"),0)
                                       .when(col("iso_code").equalTo("PNG")&&(col("date")>"2020-07-19" && col("date")<"2020-07-23"),0)
                                       .when(col("iso_code").equalTo("SVK")&&(col("date")>"2020-03-17" && col("date")<"2020-03-23"),0).otherwise(col("total_deaths")))
                            .withColumn("new_deaths_correct", 
                                        when(col("iso_code").equalTo("ISL")&&(col("date")>"2020-03-13" && col("date")<"2020-03-22"),0)
                                       .when(col("iso_code").equalTo("PNG")&&(col("date")>"2020-07-19" && col("date")<"2020-07-23"),0)
                                       .when(col("iso_code").equalTo("SVK")&&(col("date")>"2020-03-17" && col("date")<"2020-03-23"),0).otherwise(col("new_deaths")))
                            .withColumn("new_deaths_smoothed_correct", 
                                        when(col("iso_code").equalTo("ISL")&&(col("date")>"2020-03-13" && col("date")<"2020-03-22"),0)
                                       .when(col("iso_code").equalTo("PNG")&&(col("date")>"2020-07-19" && col("date")<"2020-07-23"),0)
                                       .when(col("iso_code").equalTo("SVK")&&(col("date")>"2020-03-17" && col("date")<"2020-03-23"),0).otherwise(col("new_deaths_smoothed")))
                            .cache
df_fillNullForTotalCases.unpersist()

val df_cleaned = df_fillNullForTotalDeathsSpecial
                                  .drop("new_deaths", "total_deaths", "new_deaths_smoothed") // drop old column to rename
                                  .withColumnRenamed("new_deaths_correct","new_deaths")
                                  .withColumnRenamed("total_deaths_correct","total_deaths")
                                  .withColumnRenamed("new_deaths_smoothed_correct","new_deaths_smoothed")
                                  .na.fill(0, Array("total_deaths"))
                                  .select(df.columns.head, df.columns.tail: _*)
                                  .cache
df_fillNullForTotalDeathsSpecial.unpersist()

display(df_cleaned)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
import org.apache.spark.sql.functions._

for (c <- df_cleaned.columns) {
  println(c + ": " + df_cleaned.filter(col(c).isNull).count())
}

  

>     iso_code: 0
>     continent: 0
>     location: 0
>     date: 0
>     total_cases: 0
>     new_cases: 0
>     new_cases_smoothed: 0
>     total_deaths: 0
>     new_deaths: 0
>     new_deaths_smoothed: 0
>     total_cases_per_million: 8607
>     new_cases_per_million: 0
>     new_cases_smoothed_per_million: 950
>     total_deaths_per_million: 16447
>     new_deaths_per_million: 0
>     new_deaths_smoothed_per_million: 950
>     reproduction_rate: 20054
>     icu_patients: 55381
>     icu_patients_per_million: 55381
>     hosp_patients: 54394
>     hosp_patients_per_million: 54394
>     weekly_icu_admissions: 59643
>     weekly_icu_admissions_per_million: 59643
>     weekly_hosp_admissions: 59356
>     weekly_hosp_admissions_per_million: 59356
>     total_tests: 36288
>     new_tests: 36426
>     total_tests_per_thousand: 36288
>     new_tests_per_thousand: 36426
>     new_tests_smoothed: 33726
>     new_tests_smoothed_per_thousand: 33726
>     tests_per_case: 34893
>     positive_rate: 35484
>     tests_units: 60104
>     stringency_index: 7470
>     population: 0
>     population_density: 1580
>     median_age: 2844
>     aged_65_older: 3792
>     aged_70_older: 3160
>     gdp_per_capita: 2844
>     extreme_poverty: 20871
>     cardiovasc_death_rate: 2212
>     diabetes_prevalence: 1580
>     female_smokers: 15176
>     male_smokers: 15808
>     handwashing_facilities: 30057
>     hospital_beds_per_thousand: 7269
>     life_expectancy: 316
>     human_development_index: 2528
>     import org.apache.spark.sql.functions._

  

### 3. select invariant (during pandemic) features for clustering

double check whether they are constant for each country, and if not,
change all the value to mean and filter out countries that have missing
constant features

Candidate list: - population - population*density - median*age -
aged*65*older - aged*70*older - gdp*per*capita

-   cardiovasc*death*rate
-   diabetes\_prevalence
-   female\_smokers
-   male\_smokers
-   hospital*beds*per\_thousand
-   life\_expectancy
-   human*development*index

In [ ]:
val df_invariantFeatures = df_cleaned.select($"location", $"population",$"population_density",
                                             $"median_age", $"aged_65_older",
                                             $"aged_70_older",$"gdp_per_capita",
                                             $"cardiovasc_death_rate",$"diabetes_prevalence",
                                             $"female_smokers",$"male_smokers",$"hospital_beds_per_thousand",
                                             $"life_expectancy",$"human_development_index")
display(df_invariantFeatures)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
display(df_invariantFeatures.describe())

  

[TABLE]

Truncated to 12 cols

In [ ]:
for (c <- df_invariantFeatures.columns) {
  println(c + ": " + df_invariantFeatures.filter(col(c).isNull).count())
}

  

>     location: 0
>     population: 0
>     population_density: 1580
>     median_age: 2844
>     aged_65_older: 3792
>     aged_70_older: 3160
>     gdp_per_capita: 2844
>     cardiovasc_death_rate: 2212
>     diabetes_prevalence: 1580
>     female_smokers: 15176
>     male_smokers: 15808
>     hospital_beds_per_thousand: 7269
>     life_expectancy: 316
>     human_development_index: 2528

  

Although some countries seems like an outlier, it does have constant
female*smokers and male*smokers

In [ ]:
val constant_feature_checker = df_cleaned.groupBy("location")
          .agg(
              //stddev("stringency_index").as("std_si"),         
              stddev("population").as("std_pop"),           
              stddev("population_density").as("std_pd"),
              stddev("median_age").as("std_ma"),         
              stddev("aged_65_older").as("std_a65"),           
              stddev("aged_70_older").as("std_a70"),  
              stddev("gdp_per_capita").as("std_gdp"),
              stddev("cardiovasc_death_rate").as("std_cdr"),         
              stddev("diabetes_prevalence").as("std_dp"),           
              stddev("female_smokers").as("std_fs"),      
              stddev("male_smokers").as("std_ms"),        
              stddev("hospital_beds_per_thousand").as("std_hbpt"),           
              stddev("life_expectancy").as("std_le"),
              stddev("human_development_index").as("std_hdi")
            )
           .where(
                  (col("std_pop") > 0) || (col("std_pd") > 1e-20) || (col("std_ma") > 0) || (col("std_a65") > 0) || (col("std_a70") > 0) || (col("std_gdp") > 0 ||
                   col("std_cdr") > 0) || (col("std_dp") > 0) || (col("std_fs") > 0) || (col("std_ms") > 0) || (col("std_hbpt") > 0) || (col("std_le") > 0) || (col("std_hdi") > 0))

display(constant_feature_checker)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

Each country have some constant features always

In [ ]:
val distinct_features = df_invariantFeatures.distinct()

display(distinct_features)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

In total, 126 countries have complete features

In [ ]:
val valid_distinct_features = distinct_features.filter($"population".isNotNull && $"population_density".isNotNull && $"median_age".isNotNull && 
                         $"aged_65_older".isNotNull && $"aged_70_older".isNotNull && $"gdp_per_capita".isNotNull &&
                         $"cardiovasc_death_rate".isNotNull && $"diabetes_prevalence".isNotNull && $"female_smokers".isNotNull && 
                         $"male_smokers".isNotNull && $"hospital_beds_per_thousand".isNotNull && $"life_expectancy".isNotNull &&
                         $"human_development_index".isNotNull)
display(valid_distinct_features)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

country list

In [ ]:
display(valid_distinct_features.select($"location"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
valid_distinct_features.select($"location").count()

  

>     res69: Long = 126

In [ ]:
val df_cleaned_feature = df_cleaned.filter($"location".isin(valid_distinct_features.select($"location").rdd.map(r => r(0)).collect().toSeq: _*))

display(df_cleaned_feature)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

  

### All data contains complete list of invariant time feature

(All code above is for illustration, for processing just run cell below
)

In [ ]:
// select invariant features
val df_invariantFeatures = df_cleaned.select($"location", $"population",$"population_density",
                                             $"median_age", $"aged_65_older",
                                             $"aged_70_older",$"gdp_per_capita",
                                             $"cardiovasc_death_rate",$"diabetes_prevalence",
                                             $"female_smokers",$"male_smokers",$"hospital_beds_per_thousand",
                                             $"life_expectancy",$"human_development_index").cache

// Extract valid distrinct features RDD
val valid_distinct_features = df_invariantFeatures.distinct()
                                 .filter($"population".isNotNull && $"population_density".isNotNull && $"median_age".isNotNull && 
                                 $"aged_65_older".isNotNull && $"aged_70_older".isNotNull && $"gdp_per_capita".isNotNull &&
                                 $"cardiovasc_death_rate".isNotNull && $"diabetes_prevalence".isNotNull && $"female_smokers".isNotNull && 
                                 $"male_smokers".isNotNull && $"hospital_beds_per_thousand".isNotNull && $"life_expectancy".isNotNull &&
                                 $"human_development_index".isNotNull).cache

df_invariantFeatures.unpersist()

// filter out NULL feature countries
val df_cleaned_feature = df_cleaned.filter($"location".isin(valid_distinct_features.select($"location").rdd.map(r => r(0)).collect().toSeq: _*)).cache

df_cleaned.unpersist()

display(df_cleaned_feature)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
import org.apache.spark.sql.functions._

for (c <- df_cleaned_feature.columns) {
  println(c + ": " + df_cleaned_feature.filter(col(c).isNull).count())
}

  

>     iso_code: 0
>     continent: 0
>     location: 0
>     date: 0
>     total_cases: 0
>     new_cases: 0
>     new_cases_smoothed: 0
>     total_deaths: 0
>     new_deaths: 0
>     new_deaths_smoothed: 0
>     total_cases_per_million: 4677
>     new_cases_per_million: 0
>     new_cases_smoothed_per_million: 630
>     total_deaths_per_million: 9173
>     new_deaths_per_million: 0
>     new_deaths_smoothed_per_million: 630
>     reproduction_rate: 10911
>     icu_patients: 35149
>     icu_patients_per_million: 35149
>     hosp_patients: 34162
>     hosp_patients_per_million: 34162
>     weekly_icu_admissions: 39411
>     weekly_icu_admissions_per_million: 39411
>     weekly_hosp_admissions: 39124
>     weekly_hosp_admissions_per_million: 39124
>     total_tests: 20010
>     new_tests: 20437
>     total_tests_per_thousand: 20010
>     new_tests_per_thousand: 20437
>     new_tests_smoothed: 18170
>     new_tests_smoothed_per_thousand: 18170
>     tests_per_case: 19063
>     positive_rate: 19511
>     tests_units: 39872
>     stringency_index: 2223
>     population: 0
>     population_density: 0
>     median_age: 0
>     aged_65_older: 0
>     aged_70_older: 0
>     gdp_per_capita: 0
>     extreme_poverty: 10758
>     cardiovasc_death_rate: 0
>     diabetes_prevalence: 0
>     female_smokers: 0
>     male_smokers: 0
>     handwashing_facilities: 22788
>     hospital_beds_per_thousand: 0
>     life_expectancy: 0
>     human_development_index: 0
>     import org.apache.spark.sql.functions._

  

### 4. Imputing missing time series data of

-   total*cases*per\_million
-   new*cases*per\_million
-   new*cases*smoothed*per*million
-   total*deaths*per\_million
-   new*deaths*per\_million
-   new*deaths*smoothed*per*million

In [ ]:
val per_million_data = df_cleaned_feature.select($"location", $"date", $"iso_code", $"total_cases", 
                                                 $"total_deaths", $"new_cases", $"new_deaths", $"new_cases_smoothed", 
                                                 $"new_deaths_smoothed", $"population", $"population_density", 
                                                 $"total_cases_per_million", $"new_cases_per_million", $"new_cases_smoothed_per_million", 
                                                 $"total_deaths_per_million", $"new_deaths_per_million", $"new_deaths_smoothed_per_million")

display(per_million_data)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
val per_million_data_corrected = per_million_data.withColumn("total_cases_per_million_correct", per_million_data("total_cases")/per_million_data("population")*1000000)
                                                 .withColumn("new_cases_per_million_correct", per_million_data("new_cases")/per_million_data("population")*1000000)
                                                 .withColumn("new_cases_smoothed_per_million_correct", per_million_data("new_cases_smoothed")/per_million_data("population")*1000000)
                                                 .withColumn("total_deaths_per_million_correct", per_million_data("total_deaths")/per_million_data("population")*1000000)
                                                 .withColumn("new_deaths_per_million_correct", per_million_data("new_deaths")/per_million_data("population")*1000000)
                                                 .withColumn("new_deaths_smoothed_per_million_correct", per_million_data("new_deaths_smoothed")/per_million_data("population")*1000000)
                                                 .drop("total_cases_per_million", "new_cases_per_million", "new_cases_smoothed_per_million", 
                                                       "total_deaths_per_million", "new_deaths_per_million", "new_deaths_smoothed_per_million") // drop old column to rename
                                                 .withColumnRenamed("total_cases_per_million_correct","total_cases_per_million")
                                                 .withColumnRenamed("new_cases_per_million_correct","new_cases_per_million")
                                                 .withColumnRenamed("new_cases_smoothed_per_million_correct","new_cases_smoothed_per_million")
                                                 .withColumnRenamed("total_deaths_per_million_correct","total_deaths_per_million")
                                                 .withColumnRenamed("new_deaths_per_million_correct","new_deaths_per_million")
                                                 .withColumnRenamed("new_deaths_smoothed_per_million_correct","new_deaths_smoothed_per_million")

  

>     per_million_data_corrected: org.apache.spark.sql.DataFrame = [location: string, date: string ... 15 more fields]

In [ ]:
val df_cleaned_feature_permillion = df_cleaned_feature.withColumn("total_cases_per_million_correct", df_cleaned_feature("total_cases")/df_cleaned_feature("population")*1000000)
                                                   .withColumn("new_cases_per_million_correct", df_cleaned_feature("new_cases")/df_cleaned_feature("population")*1000000)
                                                   .withColumn("new_cases_smoothed_per_million_correct", df_cleaned_feature("new_cases_smoothed")/df_cleaned_feature("population")*1000000)
                                                   .withColumn("total_deaths_per_million_correct", df_cleaned_feature("total_deaths")/df_cleaned_feature("population")*1000000)
                                                   .withColumn("new_deaths_per_million_correct", df_cleaned_feature("new_deaths")/df_cleaned_feature("population")*1000000)
                                                   .withColumn("new_deaths_smoothed_per_million_correct", df_cleaned_feature("new_deaths_smoothed")/df_cleaned_feature("population")*1000000)
                                                   .drop("total_cases_per_million", "new_cases_per_million", "new_cases_smoothed_per_million", 
                                                         "total_deaths_per_million", "new_deaths_per_million", "new_deaths_smoothed_per_million") // drop old column to rename
                                                   .withColumnRenamed("total_cases_per_million_correct","total_cases_per_million")
                                                   .withColumnRenamed("new_cases_per_million_correct","new_cases_per_million")
                                                   .withColumnRenamed("new_cases_smoothed_per_million_correct","new_cases_smoothed_per_million")
                                                   .withColumnRenamed("total_deaths_per_million_correct","total_deaths_per_million")
                                                   .withColumnRenamed("new_deaths_per_million_correct","new_deaths_per_million")
                                                   .withColumnRenamed("new_deaths_smoothed_per_million_correct","new_deaths_smoothed_per_million")

  

>     df_cleaned_feature_permillion: org.apache.spark.sql.DataFrame = [iso_code: string, continent: string ... 48 more fields]

  

### 5. Impute time series of

-   reproduction\_rate
-   total\_tests
-   stringency\_index
-   total*tests*per\_thousand

fill null in reproduction\_rate by last available and next available
value

All countries has missing data at beginning or in the end

In [ ]:
display(df_cleaned_feature_permillion.select($"reproduction_rate", $"location", $"date").filter($"reproduction_rate".isNull).groupBy("location").count().sort("location"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
display(df_cleaned_feature_permillion.select($"reproduction_rate", $"location", $"date").filter($"reproduction_rate".isNull).groupBy("location").agg(max("date").as("max_date"), min("date").as("min_date")).sort("location"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
display(df_cleaned_feature_permillion.select($"reproduction_rate", $"location", $"date").filter($"location"==="Albania"))

  

[TABLE]

Truncated to 30 rows

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val df_cleaned_reproduction_rate= df_cleaned_feature_permillion.select($"reproduction_rate", $"location", $"date")
                              .withColumn("reproduction_rate", last("reproduction_rate", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(-df_cleaned_feature_permillion.count(), 0)))
                              .withColumn("reproduction_rate", first("reproduction_rate", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(0, df_cleaned_feature_permillion.count())))
                              .na.fill(0, Array("reproduction_rate"))

  

>     import org.apache.spark.sql.functions._
>     import org.apache.spark.sql.expressions.Window
>     df_cleaned_reproduction_rate: org.apache.spark.sql.DataFrame = [reproduction_rate: double, location: string ... 1 more field]

  

countries miss stringency\_index value

In [ ]:
display(df_cleaned_feature_permillion.select($"stringency_index", $"location", $"date").filter($"stringency_index".isNull).groupBy("location").count().sort("count"))

  

[TABLE]

Truncated to 30 rows

  

start and end date for null value of stringency\_index for each country

In [ ]:
display(df_cleaned_feature_permillion.select($"stringency_index", $"location", $"date").filter($"stringency_index".isNull).groupBy("location").agg(max("date").as("max_date"), min("date").as("min_date")))

  

[TABLE]

Truncated to 30 rows

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val df_cleaned_stringency = df_cleaned_feature_permillion.select($"stringency_index", $"location", $"date")
                              .withColumn("stringency_index_corect", last("stringency_index", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(-df_cleaned_feature_permillion.count(), 0)))
display(df_cleaned_stringency.filter($"stringency_index".isNull).filter($"stringency_index_corect".isNull).groupBy("location").count())

  

[TABLE]

  

total\_tests, impute by last available or next available value

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val df_cleaned_total_cases = df_cleaned_feature_permillion.select($"total_tests", $"location", $"date")
                              .withColumn("total_tests", last("total_tests", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(-df_cleaned_feature_permillion.count(), 0)))
                              .withColumn("total_tests", first("total_tests", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(0, df_cleaned_feature_permillion.count())))
                              .na.fill(0, Array("total_tests"))

  

>     import org.apache.spark.sql.functions._
>     import org.apache.spark.sql.expressions.Window
>     df_cleaned_total_cases: org.apache.spark.sql.DataFrame = [total_tests: double, location: string ... 1 more field]

In [ ]:
display(df_cleaned_feature_permillion.select($"total_tests", $"location", $"date").filter($"total_tests".isNull).groupBy("location").count())

  

[TABLE]

Truncated to 30 rows

In [ ]:
val total_tests_date_maxmin = df_cleaned_feature_permillion.select($"total_tests", $"location", $"date").filter($"total_tests".isNull).groupBy("location").agg(max("date").as("max_date"), min("date").as("min_date"))
display(total_tests_date_maxmin)

  

[TABLE]

Truncated to 30 rows

  

process stringency*index, reproduction*rate, total*tests,
total*tests*per*thousand

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val df_cleaned_time_series = df_cleaned_feature_permillion
                              .withColumn("reproduction_rate", last("reproduction_rate", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(-df_cleaned_feature_permillion.count(), 0)))
                              .withColumn("reproduction_rate", first("reproduction_rate", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(0, df_cleaned_feature_permillion.count())))
                              .na.fill(0, Array("reproduction_rate"))
                              .withColumn("stringency_index", last("stringency_index", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(-df_cleaned_feature_permillion.count(), 0)))
                              .na.fill(0, Array("stringency_index"))
                              .withColumn("total_tests", last("total_tests", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(-df_cleaned_feature_permillion.count(), 0)))
                              .withColumn("total_tests", first("total_tests", true)
                                         .over(Window.partitionBy("location").orderBy("date").rowsBetween(0, df_cleaned_feature_permillion.count())))
                              .na.fill(0, Array("total_tests"))
                              .withColumn("total_tests_per_thousand", col("total_tests")/col("population")*1000)


  

>     import org.apache.spark.sql.functions._
>     import org.apache.spark.sql.expressions.Window
>     df_cleaned_time_series: org.apache.spark.sql.DataFrame = [iso_code: string, continent: string ... 48 more fields]

In [ ]:
display(df_cleaned_time_series)

  

[TABLE]

Truncated to 30 rows

Truncated to 12 cols

In [ ]:
import org.apache.spark.sql.functions._

for (c <- df_cleaned_time_series.columns) {
  println(c + ": " + df_cleaned_time_series.filter(col(c).isNull).count())
}

  

>     iso_code: 0
>     continent: 0
>     location: 0
>     date: 0
>     total_cases: 0
>     new_cases: 0
>     new_cases_smoothed: 0
>     total_deaths: 0
>     new_deaths: 0
>     new_deaths_smoothed: 0
>     reproduction_rate: 0
>     icu_patients: 35149
>     icu_patients_per_million: 35149
>     hosp_patients: 34162
>     hosp_patients_per_million: 34162
>     weekly_icu_admissions: 39411
>     weekly_icu_admissions_per_million: 39411
>     weekly_hosp_admissions: 39124
>     weekly_hosp_admissions_per_million: 39124
>     total_tests: 0
>     new_tests: 20437
>     total_tests_per_thousand: 0
>     new_tests_per_thousand: 20437
>     new_tests_smoothed: 18170
>     new_tests_smoothed_per_thousand: 18170
>     tests_per_case: 19063
>     positive_rate: 19511
>     tests_units: 39872
>     stringency_index: 0
>     population: 0
>     population_density: 0
>     median_age: 0
>     aged_65_older: 0
>     aged_70_older: 0
>     gdp_per_capita: 0
>     extreme_poverty: 10758
>     cardiovasc_death_rate: 0
>     diabetes_prevalence: 0
>     female_smokers: 0
>     male_smokers: 0
>     handwashing_facilities: 22788
>     hospital_beds_per_thousand: 0
>     life_expectancy: 0
>     human_development_index: 0
>     total_cases_per_million: 0
>     new_cases_per_million: 0
>     new_cases_smoothed_per_million: 0
>     total_deaths_per_million: 0
>     new_deaths_per_million: 0
>     new_deaths_smoothed_per_million: 0
>     import org.apache.spark.sql.functions._